# AnoGAN

In [ ]:
from model.dcgan import DCGAN
from model.anogan import AnoGAN

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D, LeakyReLU, Flatten, Dense, Reshape, Conv2DTranspose, Input, MaxPool2D, UpSampling2D, BatchNormalization, Activation, Dropout, ZeroPadding2D
%matplotlib inline

In [ ]:
EPOCHS = 50
NOISE_DIM = 100

IMAGE_SIZE = (64, 64)
CHANNELS = 3
IMAGE_SHAPE = IMAGE_SIZE + tuple([CHANNELS])

## Dataset

In [ ]:
TRAIN_PATH = r"C:\Users\filip\Desktop\Universita\Laboratorio di Intelligenza Artificiale\hazelnut\train"
TEST_PATH = r"C:\Users\filip\Desktop\Universita\Laboratorio di Intelligenza Artificiale\hazelnut\test"

training_set = tf.keras.utils.image_dataset_from_directory(TRAIN_PATH, image_size = IMAGE_SIZE, batch_size = None, shuffle = False)
test_set = tf.keras.utils.image_dataset_from_directory(TEST_PATH, image_size = IMAGE_SIZE)

### Normalizzazione

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(scale = (1./127.5), offset = -1)

normalized_training_set = training_set.map(lambda x, y: (normalization_layer(x), y))
normalized_test_set = test_set.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
BUFFER_SIZE = 391
BATCH_SIZE = 17
normalized_training_set = normalized_training_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
len(normalized_training_set)

## GAN

In [ ]:
gan = DCGAN(IMAGE_SHAPE, NOISE_DIM)

In [ ]:
img = gan.generator(tf.random.normal([1,100]))

plt.imshow(img[0, :, :, 0])
plt.show()

In [ ]:
print(gan.discriminator(img))

## Training

In [ ]:
#gan.restore()
disc_loss, gen_loss = gan.train(normalized_training_set, EPOCHS)

In [ ]:
generated_img = gan.generator(tf.random.normal([1, 100]))
i = np.array((generated_img[0] * 127.5) + 127.5 , np.int32)
plt.imshow(i)
plt.show()

In [ ]:
plt.plot(disc_loss)
plt.plot(gen_loss)
plt.show()

## Anomaly Detection

In [ ]:
anogan = AnoGAN(gan)

In [ ]:
test_image = []
for images, labels in normalized_test_set:
    for i in range(1):
        test_image = images[i].numpy()
    break
test_image = np.array([test_image])
anomaly_score, similar_img = anogan.anomaly_detection(test_image)

In [ ]:
print(anomaly_score)

In [ ]:
t = np.array((test_image[0] * 127.5) + 127.5, np.int32)
s = np.array((similar_img[0] * 127.5) + 127.5, np.int32)
plt.imshow(abs(t-s))
plt.show()

In [ ]:
plt.imshow(abs(similar_img[0, :, :, 0]-test_image[0, : ,: ,0]), cmap="Accent_r")

In [ ]:
plt.imshow(t, cmap="Accent_r")